# AI Computer Assignment 1 (Search Algorithms)
Mohammad Saadati - 
_810198410_

## Introduction
In this project, we are going to find a suitable solution for the following problem and implement it by using uninformed search algorithms (BFS and IDS) and informed search algorithms (A* and weighted A*).

The goal of this computer assignment to help our agent to do its job. Our agent is Gandalf. Gray Gandalf is tasked with confronting Sauron, the lord of darkness, with the goal of leading and guiding the ring fellows. To do this, he must pass the fellows of the ring through the eurekas, which are Sauron's army, and bring each one to a specific position, and at the end he goes to Gondor. 

### Import Libraries

In [1]:
from time import time
import heapq
import math

### Defining Constants

In [2]:
TESTCASE1 = "test_00.txt"
TESTCASE2 = "test_01.txt"
TESTCASE3 = "test_02.txt"
TESTCASE4 = "test_03.txt"
NUMBER_OF_TEST_REPETITIONS = 3
BLANK = " "
HELPER = "h"
EUREKA = "e"
DES_HELPER = "d"
RIGHT = "R"
UP = "U"
LEFT = "L"
DOWN = "D"
TYPE = 0
ID = 1
HEURISTIC = 0
HIGH_WEIGHTED = 1
LOW_WEIGHTED = 2
SMALL_ALPHA = 2.25
BIG_ALPHA = 12.25
MAP_LENGHT = 0
MAP_WIDTH = 0
NUMBER_OF_FELLOWS = 0

## Modeling the Problem
* State: a list of the positions of the Gandalf, status of reaching the destination of each Helper, has Gandalf taken the Helper to his destination or not, is Gandalf at home under the supervision of Eureka and the military rank of the Eureka under his supervision  
* Initial State: all Helpers are left and the Gandalf is at the given starting position
* Actions: move one unit to Right, Up, Left, Down
* Transition model: it's implented in the Node class and generates a new state or false from current state and action.
* Goal State: all Helpers should have reached their destination and Gandalf should have reached his destination.

In [3]:
class Node:
	def __init__(self, gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map,
                 cur_helper, cur_eureka_area, eureka_area_cur_capacity, is_helpers_reach_des, parent=None, action=None):
		self.gandalf_x = gandalf_x
		self.gandalf_y = gandalf_y
		self.end_x = end_x
		self.end_y = end_y
		self.eureka_manhattan = eureka_manhattan
		self.des_helpers_x = des_helpers_x
		self.des_helpers_y = des_helpers_y
		self.game_map = game_map
		self.cur_helper = cur_helper
		self.cur_eureka_area = cur_eureka_area
		self.eureka_area_cur_capacity = eureka_area_cur_capacity
		self.is_helpers_reach_des = is_helpers_reach_des
		self.parent = parent
		self.action = action
		self.path_cost = 0
		if self.parent:
			self.path_cost = parent.get_cost() + 1

	def get_cost(self):
		return self.path_cost

	def goal_test(self):
		if self.gandalf_x == self.end_x and self.gandalf_y == self.end_y:
			if False in self.is_helpers_reach_des:
				return False
			else:
				return True
		return False

	def get_children(self):
		children = set()
		self.add_new_child(children, 0, 1, RIGHT)
		self.add_new_child(children, 1, 0, DOWN)
		self.add_new_child(children, 0, -1, LEFT)
		self.add_new_child(children, -1, 0, UP)
		return children

	def add_new_child(self, children, delta_x, delta_y, action):
		move = False
		i = self.gandalf_x
		j = self.gandalf_y
        
		if action == RIGHT:
			if j == len(self.game_map[0]) - 1:
				move = False
				return
		elif action == DOWN:
			if i == len(self.game_map) - 1:
				move = False
				return
		elif action == LEFT:
			if j == 0:
				move = False
				return           
		elif action == UP:
			if i == 0:
				move = False
				return 
            
		gandalf_x = self.gandalf_x
		gandalf_y = self.gandalf_y
		end_x = self.end_x
		end_y = self.end_y
		eureka_manhattan = self.eureka_manhattan[:]
		des_helpers_x = self.des_helpers_x[:]
		des_helpers_y = self.des_helpers_y[:]
		game_map = self.game_map
		cur_helper = self.cur_helper
		cur_eureka_area = self.cur_eureka_area
		eureka_area_cur_capacity = self.eureka_area_cur_capacity
		is_helpers_reach_des = self.is_helpers_reach_des[:]
        
		if game_map[i+delta_x][j+delta_y][TYPE] != EUREKA:
			eureka_area_cur_capacity = 0
			cur_eureka_area = -1
			if cur_helper != -1:
				if des_helpers_x[cur_helper] == i+delta_x and des_helpers_y[cur_helper] == j+delta_y:
					if is_helpers_reach_des[cur_helper] == 0:
						is_helpers_reach_des[cur_helper] = 1
						cur_helper = -1
			if game_map[i+delta_x][j+delta_y][TYPE] == HELPER and cur_helper == -1:
				if is_helpers_reach_des[int(game_map[i+delta_x][j+delta_y][ID])] == 0:
					cur_helper = int(game_map[i+delta_x][j+delta_y][ID])
			move = True
		else:
			if cur_eureka_area == -1:
				cur_eureka_area = int(game_map[i+delta_x][j+delta_y][ID])
				eureka_area_cur_capacity = 1
				move = True
			elif cur_eureka_area != -1 and cur_eureka_area == int(game_map[i+delta_x][j+delta_y][ID]):
				if eureka_area_cur_capacity < eureka_manhattan[cur_eureka_area]:
					eureka_area_cur_capacity += 1
					move = True
				else:
					move = False
			elif cur_eureka_area != -1 and cur_eureka_area != int(game_map[i+delta_x][j+delta_y][ID]):
				cur_eureka_area = int(game_map[i+delta_x][j+delta_y][ID])
				eureka_area_cur_capacity = 1
				move = True               

		if move == True:
			gandalf_x += delta_x
			gandalf_y += delta_y
			p = ""
			p += action
			p += " "
			new_node = Node(gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map,
                                cur_helper, cur_eureka_area, eureka_area_cur_capacity, is_helpers_reach_des, self, p)
			children.add(new_node)

	def get_string(self):
		s = ""
		for i in range(len(self.is_helpers_reach_des)):
			s += str(self.is_helpers_reach_des[i])
			s += '$'
		s += str(self.gandalf_x)
		s += '$'
		s += str(self.gandalf_y)
		s += '$'
		s += str(self.cur_helper)
		s += '$'
		s += str(self.cur_eureka_area)
		s += '$'
		s += str(self.eureka_area_cur_capacity)
		s += '$'
		return s

	def get_state(self):
		info = list()
		info.append(self.gandalf_x)
		info.append(self.gandalf_y)
		info.append(self.is_helpers_reach_des)
		info.append(self.cur_helper)
		return info

### Defining FIFO Queue

In [4]:
class FIFO_queue:
	def __init__(self):
		self.datalist = []

	def enqueue(self, data):
		self.datalist.append(data)

	def dequeue(self):
		if self.is_empty() == True:
			return
		data = self.datalist[0]
		self.datalist.pop(0)
		return data

	def is_empty(self):
		if len(self.datalist) == 0:
			return True
		return False

### BFS Algorithm 
The Breadth First Search `BFS` algorithm is an uninformed search algorithm. We use a *FIFO Queue* for BFS class as the frontier set and we expand the shallowest states first. The BFS class has an explored set that every
time we expand a node, we add its state to this set, Also we
stored number of visited states and number of unique visited
states in this class that we update them during the execution of
the algorithm. The algorithm runs when we call the *run_bfs()*
function on its instance. 
At first we add intial state into frontier and then pop it from the queue. 
Then all possible actions from initial values which does not explored before, will add to the frontier 
and next we will redo these steps until we find goal state. BFS returns the solution by the time it reaches to it.

In [5]:
class BFS:
	def __init__(self, gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map):
		self.inital_node = Node(gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map,
                                -1, -1, 0, [0]*len(des_helpers_x))
		self.frontier = FIFO_queue()
		self.explored = set()
		self.unique_states = set()
		self.number_of_states = 0
		self.number_of_unique_states = 0
		self.path_cost = 0
		self.goal_node = None
		self.path = ''
 
	def run_bfs(self):
		node = self.inital_node
		if node.goal_test():
			self.goal_node = node
			self.path_cost = node.path_cost
			return True
		self.frontier.enqueue(node)
		while not self.frontier.is_empty():
			self.number_of_states += 1
			node = self.frontier.dequeue()
			if node.get_string() in self.explored:
				continue
			self.explored.add(node.get_string())
			if not node.get_string() in self.unique_states:
				self.unique_states.add(node.get_string())
				self.number_of_unique_states += 1
			if node.goal_test():
				self.goal_node = node
				self.path_cost = node.path_cost
				return True
			for child in node.get_children():
				if not child.get_string() in self.explored:
					if child.goal_test():
						self.goal_node = child
						self.path_cost= child.path_cost
						return True
					self.frontier.enqueue(child)
		return False

	def show_bfs_result(self):
		node = self.goal_node
		while node.parent != None:
			self.path += node.action
			node = node.parent
		self.path = ''.join(reversed(self.path))
		print("$ BFS $")
		print("Path Cost: " + str(self.path_cost))
		print("Path: " + self.path)
		print("Number of Visited States: " + str(self.number_of_states))
		print("Number of Unique Visited States: " + str(self.number_of_unique_states))

### IDS Algorithm
The Iterative Deepening Search `IDS` algorithm is another uninformed search algorithm. It uses Depth Limited Search `DLS` algorithm with different max depth.    The IDS class has a *Stack (LIFO queue)* is used as the frontier set and the deepest states are expanded first, i.e. in a branch we go as deep as possible and then we traverse back up, an explored set that every
time we expand a node, we add its state to this set and a
dictionary that has each state as its key and minimum depth that
this state has been visited at. Also we stored number of visited
states and number of unique visited states in this class that we
update them during the execution of the algorithm. It may be slower than `BFS` and will just find the first answer which may not be an optimal solution to the problem. It also fails in infinite-depth spaces. The algorithm runs when we call the *run_ids()* function on its instance. This function calls the subroutine *dls* that executes a depth limited search with the specified depth. We have a loop to increas depth in every iteration of it. Actualy at every iteration we increase depth and run inner loop which is a limited dfs. At every itteration we reset all of data structures to initial values. In limited dfs  We pop first node from stack. Then if it's depth is our limit we didn't add it's children to frontier else we add chidlren to frontier

In [6]:
class IDS:
	def __init__(self, gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map):
		self.inital_node = Node(gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map,
                                -1, -1, 0, [0]*len(des_helpers_x))
		self.explored = set()
		self.unique_states = set()
		self.number_of_states = 0
		self.number_of_unique_states = 0
		self.path_cost = 0
		self.goal_node = None
		self.path = ''
		self.min_depth_explored = {}

	def dls(self, node, depth):
		self.number_of_states += 1
		if not node.get_string() in self.unique_states:
			self.unique_states.add(node.get_string())
			self.number_of_unique_states += 1
		if node.goal_test():
			self.goal_node = node
			return True
		if depth <= 0:
			return False
		self.min_depth_explored[node.get_string()] = depth
		for child in node.get_children():
			if (not child.get_string() in self.min_depth_explored) or self.min_depth_explored[child.get_string()] < depth-1:
				if self.dls(child, depth-1) == True:
					return True	
		return False

	def run_ids(self):
		depth = 1
		self.min_depth_explored = {}
		node = self.inital_node
		for i in range(8*MAP_LENGHT*MAP_WIDTH*NUMBER_OF_FELLOWS):
			if self.dls(node, depth) == True:
				self.path_cost = depth
				return True
			depth +=1
		
	def show_ids_result(self):
		node = self.goal_node
		while node.parent != None:
			self.path += node.action
			node = node.parent
		print("$ IDS $")
		print("Path Cost: " + str(self.path_cost))
		print("Path: " + self.path)
		print("Number of Visited States: " + str(self.number_of_states))
		print("Number of Unique Visited States: " + str(self.number_of_unique_states))

### Defining HEAP Queue

In [7]:
class HEAP_queue:
    def __init__(self):
        self.top = 0
        self.datalist = []
        
    def enqueue(self, data, indicator):
        self.top += 1
        heapq.heappush(self.datalist, (indicator, self.top, data))
        
    def dequeue(self):
        return heapq.heappop(self.datalist)[-1]

    def is_empty(self):
        if len(self.datalist) == 0:
            return True
        return False

### A* Algorithm
This algorithm is very similar to `BFS` algorithm, but it uses a priority queue as it's frontier instead. The A* class 
has a frontier set that is implemented by min-heap queue that its indicator for choosing a node is the value of the evaluating
function *f*. The argument of this function is a node and the return value of this function is sum of the path_cost of the node
and the value of the heuristic function. At first we add intial state into frontier and then pop it from the queue. Then all possible actions from initial values which does not explored before, will add to the frontier and next we will redo these steps until we find goal state.

#### Heuristic:
 
- When all the Helpers have reached their destination, the heuristic is the Manhattan distance from Gandalf's current location to Gandalf's final destination.

- When Gandalf picks a Helper to reach his destination, the heuristic is the Manhattan distance from Gandalf's current location to the destination of the Helper he picks, plus the Manhattan distance from the destination of the Helper he delivered to Gandalf's final destination.

- When Gandalf has not selected a helper to reach his destination, the heuristic is the maximum of Manhattan distance of Gandalf's current location to the origin of any helper that has not reached its destination, plus the Manhattan distance from the origin of any remaining helper to its destination, plus the Manhattan distance from Helper's destination to Gandalf's final destination.

#### Proof of Amissiblity: 
This heuristic never overestimates the cost to reach the goal, because in the first case the heuristic is less than the actual distance to the destination, and in the second and third cases we have at least one Helper from there, and we only calculate the distance of one Helper. So again it is less than the actual distance.

$$ h(n) \leqslant \text{h*}(n) $$ 

#### Proof of Consistency: 
If the following equation holds, a heuristic is consistant:

$$ cost(A to C) \geq h(A) - h(C)$$

We need to check for every possible move the difference of the heuristics is less than or equal to one:

- Gandalf moves a Helper closer to the his destination:
    In this case the the Helper moves one step closer so the value will also be less than or equal to one.

- Gandalf moves without moving any Helpers:
    In this case the heusitics for state C and A are equal therfore the difference is equal to zero and the quation holds.
    
- Gandalf moves a Helper further away from the his destination:
    In this case the heuristic of the later state is larger so the value will become negative so the equation holds.    


### Weighted A*
The idea of this search is to speed up search at the expense of optimality the difference is that we use weight and multiply it by heuristic   

f = g + alpha * heuristic
  
and the heuristic is no longer admissible because first solution found is not guaranteed to be optimal.

In [8]:
class A_STAR:
	def __init__(self, gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map, mode
                 , helpers_x, helpers_y):
		self.inital_node = Node(gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map,
                                -1, -1, 0, [0]*len(des_helpers_x))
		self.frontier = HEAP_queue()
		self.explored = set()
		self.unique_states = set()
		self.number_of_states = 0
		self.number_of_unique_states = 0
		self.path_cost = 0
		self.goal_node = None
		self.path = ''
		self.mode = mode
		self.helpers_x = helpers_x
		self.helpers_y = helpers_y
		self.des_helpers_x = des_helpers_x
		self.des_helpers_y = des_helpers_y
		self.end_x = end_x
		self.end_y = end_y
        
	def f(self, node):
		if self.mode == HEURISTIC:
			return self.g(node) + self.h(node)
		elif self.mode == LOW_WEIGHTED:
			return self.g(node) + SMALL_ALPHA * self.h(node)
		elif self.mode == HIGH_WEIGHTED:
			return self.g(node) + BIG_ALPHA * self.h(node)

	def g(self, node):
		return node.path_cost

	def h(self, node):
		sum_distance = 0
		min_distance = math.inf
		gandalf_x = node.get_state()[0]
		gandalf_y = node.get_state()[1]
		is_helpers_reach_des = node.get_state()[2]
		cur_helper = node.get_state()[3]
		if cur_helper != -1:
			sum_distance = abs(gandalf_x - self.des_helpers_x[cur_helper]) + abs(gandalf_y - self.des_helpers_y[cur_helper])
			sum_distance += abs(self.end_x - self.des_helpers_x[cur_helper]) + abs(self.end_y - self.des_helpers_y[cur_helper])
		else:
			for i in range(len(is_helpers_reach_des)):
				if is_helpers_reach_des[i] == False: 
					manhattan = abs(gandalf_x - self.helpers_x[i]) + abs(gandalf_y - self.helpers_y[i]) + abs(self.helpers_x[i] - self.des_helpers_x[i]) + abs(self.helpers_y[i] - self.des_helpers_y[i])
					manhattan += abs(self.end_x - self.des_helpers_x[i]) + abs(self.end_y - self.des_helpers_y[i])
					if manhattan < min_distance:
						min_distance = manhattan                       
					sum_distance = min_distance
		if not False in is_helpers_reach_des:
			sum_distance = abs(gandalf_x - self.end_x) + abs(gandalf_y - self.end_y)
		return sum_distance

	def run_a_star(self):
		node = self.inital_node
		if node.goal_test():
			self.goal_node = node
			self.path_cost= node.path_cost
			return True
		self.frontier.enqueue(node, self.f(node))
		while not self.frontier.is_empty():
			self.number_of_states += 1
			node = self.frontier.dequeue()
			if node.get_string() in self.explored:
				continue
			self.explored.add(node.get_string())
			if not node.get_string() in self.unique_states:
				self.unique_states.add(node.get_string())
				self.number_of_unique_states += 1
			if node.goal_test():
				self.goal_node = node
				self.path_cost= node.path_cost
				return True
			for child in node.get_children():
				if not child.get_string() in self.explored:
					self.frontier.enqueue(child, self.f(child))
		return False

	def show_a_star_result(self):
		node = self.goal_node
		while node.parent != None:
			self.path += node.action
			node = node.parent
		self.path = ''.join(reversed(self.path))
		if self.mode == HEURISTIC:
			print("$ A* Search $")
		elif self.mode == LOW_WEIGHTED:
			print("$ Weighted A* Search with Alpha = " + str(SMALL_ALPHA) + " $")
		elif self.mode == HIGH_WEIGHTED:
			print("$ Weighted A* Search with Alpha = " + str(BIG_ALPHA) + " $")   
		print("Path Cost: " + str(self.path_cost))
		print("Path: " + self.path)
		print("Number of Visited States: " + str(self.number_of_states))
		print("Number of Unique Visited States: " + str(self.number_of_unique_states))

### Importing Data
In this part, the inputs read from the file and splited and stored in lists and variables.

In [9]:
def read_data(filename):
	n = 0
	m = 0
	gandalf_x = -1 
	gandalf_y = -1
	end_x = -1
	end_y = -1
	k = 0
	l = 0
	global MAP_LENGHT
	global MAP_WIDTH
	global NUMBER_OF_FELLOWS
	eurekas_x = list()
	eurekas_y = list()
	eureka_manhattan = list()
	helpers_x = list()
	helpers_y = list()
	des_helpers_x = list()
	des_helpers_y = list()
	game_map = []
	with open(filename) as f:
		n, m = [int(x) for x in next(f).split(' ')]
		MAP_LENGHT = n
		MAP_WIDTH = m       
		gandalf_x, gandalf_y = [int(x) for x in next(f).split(' ')]
		end_x, end_y = [int(x) for x in next(f).split(' ')]
		k, l = [int(x) for x in next(f).split(' ')]
		NUMBER_OF_FELLOWS = l
		for i in range(0, n):
			game_map.append([BLANK for i in range(m)])
		for i in range(k):
			eureka_x, eureka_y, eureka_man = [int(x) for x in next(f).split(' ')] 
			for j in range(eureka_man):
				for k in range(j * 2 + 1):
					if eureka_x - eureka_man + j >= 0 and eureka_y - j + k >=0 and eureka_y - j + k < m:
						game_map[eureka_x - eureka_man + j][eureka_y - j + k] = EUREKA + str(i)
			for j in range(eureka_man):
				for k in range(j * 2 + 1):
					if eureka_x + eureka_man - j < n and eureka_y - j + k >=0 and eureka_y - j + k < m:
						game_map[eureka_x + eureka_man - j][eureka_y - j + k] = EUREKA + str(i)
			for j in range(eureka_man * 2 + 1):
				if eureka_y - eureka_man + j >=0 and eureka_y - eureka_man + j < m:
					game_map[eureka_x][eureka_y - eureka_man + j] = EUREKA + str(i)
			eureka_manhattan.append(eureka_man)
		for i in range(l):
			helper_x, helper_y = [int(x) for x in next(f).split(' ')]
			game_map[helper_x][helper_y] = HELPER + str(i) 
			helpers_x.append(helper_x)
			helpers_y.append(helper_y)
		for i in range(l):
			des_helper_x, des_helper_y = [int(x) for x in next(f).split(' ')] 
			des_helpers_x.append(des_helper_x)
			des_helpers_y.append(des_helper_y)

	return gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map, helpers_x, helpers_y

### Running BFS

In [10]:
def run_BFS(test_file):
	gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map, helpers_x, helpers_y = read_data(test_file)
	bfs = BFS(gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map)
	average_time = 0
	for i in range(0, NUMBER_OF_TEST_REPETITIONS):
		bfs = BFS(gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map)
		start_time = time()
		bfs.run_bfs()
		finish_time = time()
		average_time += (finish_time - start_time)
	bfs.show_bfs_result()
	average_time /= NUMBER_OF_TEST_REPETITIONS 
	print("Average Execution Time for 3 BFS runs: " + str(average_time) + " seconds\n")

### Running IDS

In [11]:
def run_IDS(test_file):   
	gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map, helpers_x, helpers_y = read_data(test_file)
	ids = IDS(gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map)
	average_time = 0
	for i in range(0, NUMBER_OF_TEST_REPETITIONS):
		ids = IDS(gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map)
		start_time = time()
		ids.run_ids()
		finish_time = time()
		average_time += (finish_time - start_time)
	ids.show_ids_result()
	average_time /= NUMBER_OF_TEST_REPETITIONS
	print("Average Execution Time for 3 IDS runs: " + str(average_time) + " seconds\n")

### Running A*

In [12]:
def run_A_STAR(test_file, version):
	gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map, helpers_x, helpers_y = read_data(test_file)
	a_star = A_STAR(gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map, version, helpers_x, helpers_y)
	average_time = 0
	for i in range(0, NUMBER_OF_TEST_REPETITIONS):
		a_star = A_STAR(gandalf_x, gandalf_y, end_x, end_y, eureka_manhattan, des_helpers_x, des_helpers_y, game_map, version, helpers_x, helpers_y)
		start_time = time()
		a_star.run_a_star()
		finish_time = time()
		average_time += (finish_time - start_time)
	a_star.show_a_star_result()
	average_time /= NUMBER_OF_TEST_REPETITIONS
	print("Average Execution Time for 3 A* runs: " + str(average_time) + " seconds\n")

### Running Testcases

In [13]:
def run_test(test_file):
	run_BFS(test_file)
	run_IDS(test_file)
	run_A_STAR(test_file, HEURISTIC)
	run_A_STAR(test_file, LOW_WEIGHTED)
	run_A_STAR(test_file, HIGH_WEIGHTED)

In [14]:
print("Testcase #1....\n")
run_test(TESTCASE1)

Testcase #1....

$ BFS $
Path Cost: 48
Path:  R R R R R R R R R R R R R R R R R R R R R R R R R R R R R D U R D R U D R U D U R D R R R R R R
Number of Visited States: 10506
Number of Unique Visited States: 6426
Average Execution Time for 3 BFS runs: 0.1825258731842041 seconds

$ IDS $
Path Cost: 48
Path: R R R R R R D U R D U R D U R D U R D R R R R R R R R R R R R R R R R R R R R R R R R R R R R R 
Number of Visited States: 50360
Number of Unique Visited States: 7279
Average Execution Time for 3 IDS runs: 1.0537381966908772 seconds

$ A* Search $
Path Cost: 48
Path:  R R R R R R R R R R R R R R R R R R R R R R R R R R R R R D U R D U R D R U D U R D R R R R R R
Number of Visited States: 1295
Number of Unique Visited States: 919
Average Execution Time for 3 A* runs: 0.033869663874308266 seconds

$ Weighted A* Search with Alpha = 2.25 $
Path Cost: 48
Path:  R R R R R R R R R R R R R R R R R R R R R R R R R R R R R D R U D R U D R U D U R D R R R R R R
Number of Visited States: 570
Numb

In [15]:
print("Testcase #2....\n")
run_test(TESTCASE2)

Testcase #2....

$ BFS $
Path Cost: 52
Path:  R R D D R R L L D D D D D D L R R U U R U U U R R L L D D D L D D R R R R R U U U U D D D L D L L R R R
Number of Visited States: 2022
Number of Unique Visited States: 1381
Average Execution Time for 3 BFS runs: 0.02969646453857422 seconds

$ IDS $
Path Cost: 52
Path: R R R L L D L D D D R R R R U U U R U L D D D D D L L U R U R U U R R R U L D D D D D L D L R R D R D R 
Number of Visited States: 31390
Number of Unique Visited States: 1395
Average Execution Time for 3 IDS runs: 0.6171418825785319 seconds

$ A* Search $
Path Cost: 52
Path:  R D R D R R L D L D D D D D L U R U R R U U R R U D L L D D L D D U U R U R U R R R D D D L D L L R R R
Number of Visited States: 1435
Number of Unique Visited States: 991
Average Execution Time for 3 A* runs: 0.02336772282918294 seconds

$ Weighted A* Search with Alpha = 2.25 $
Path Cost: 54
Path:  R R D D R R L L D D D D D L D R R R U U U R U R U D L L D D L D D R U U U U R R R R L L L D L D D R D R R R

In [16]:
print("Testcase #3....\n")
run_test(TESTCASE3)

Testcase #3....

$ BFS $
Path Cost: 34
Path:  R R R R R R R L D L L D L D D L D L L D R D U U R U R R R D R R D D
Number of Visited States: 550
Number of Unique Visited States: 384
Average Execution Time for 3 BFS runs: 0.007004261016845703 seconds

$ IDS $
Path Cost: 34
Path: D D R D R R R U R U R U R D D L L D L D D L L D L D L R R R R R R R 
Number of Visited States: 5375
Number of Unique Visited States: 404
Average Execution Time for 3 IDS runs: 0.08536807696024577 seconds

$ A* Search $
Path Cost: 34
Path:  R R R R R R R D L L D L L D D D L L L R D D U U R R U R R R D R D D
Number of Visited States: 301
Number of Unique Visited States: 211
Average Execution Time for 3 A* runs: 0.004955450693766276 seconds

$ Weighted A* Search with Alpha = 2.25 $
Path Cost: 34
Path:  R R R R R R R D L L L D D L D D L L L R D D R U U R U R R D R R D D
Number of Visited States: 221
Number of Unique Visited States: 169
Average Execution Time for 3 A* runs: 0.0039028326670328775 seconds

$ Weighted A*

In [17]:
print("Testcase #4....\n")
run_test(TESTCASE4)

Testcase #4....

$ BFS $
Path Cost: 66
Path:  D D R D R D D D L L D D R D R R R R U U U U U U R U R U U R R L L D L D D D D D L L D L D L L D U R R U R R U U U R R R R D D D D D
Number of Visited States: 5864
Number of Unique Visited States: 3921
Average Execution Time for 3 BFS runs: 0.09383455912272136 seconds

$ IDS $
Path Cost: 66
Path: D D D D D R R R R U U U R R R U R U L L L D D D D L L D D D D L D L L R U R R U U U U U R R U U R R R U D R L L D D D D D D R R D D 
Number of Visited States: 103344
Number of Unique Visited States: 3961
Average Execution Time for 3 IDS runs: 2.0671929518381753 seconds

$ A* Search $
Path Cost: 66
Path:  D R D D R D D L D L D D R D R U R U R U U R U R U U R U R U R L L D L D D L D L L L D D D D L D U R R U R R U U U R R R R D D D D D
Number of Visited States: 3919
Number of Unique Visited States: 2649
Average Execution Time for 3 A* runs: 0.07739734649658203 seconds

$ Weighted A* Search with Alpha = 2.25 $
Path Cost: 66
Path:  R D R D D D D L D L D

## Compare algorithms with each other

#### BFS:
BFS can be fast in searching but the hold-up is that it's uninformed. Also BFS does not visit a state twice.

*Advantages:*
* The solution will definitely found out by BFS If there is some solution.
* If there is more than one solution then it will find a solution with minimal steps.
* BFS will never get trapped in a blind alley, which means unwanted nodes.

*Disadvantages:*
* If a solution is far away then it consumes time.
* Memory Constraints As it stores all the nodes of the present level to go for the next level.

BFS is optimal if cost = 1 per step (if the path cost is a nondecreasing function of the depth of the node)

#### IDS:
IDS is much slower than the other search algorithms again one reason being that IDS is uninformed but it's main advantage is it's memory. It can operate with just a stack proportional to the depth of one branch.

*Advantages:*
* When the solutions are found at the lower depths say n, then the algorithm proves to be efficient and in time.

*Disadvantages:*
* The main problem with IDS is the time and wasted calculations that take place at each depth.

IDS is optimal if step cost = 1

#### A*:
A* is a very fast search algorithm a good thing about is that A* is an informed search algorithm so this means it uses a heuristic function to search in the rational direction.

If h(n) is consistent, then the values of f(n) along any path are nondecreasing. Whenever A∗ selects a node n for expansion, the optimal path to that node has been found.

If A* returns a solution, that solution is guaranteed to be optimal, as long as:
* the branching factor is finite
* arc costs are strictly positive
* h(n) is an underestimate of the length of the shortest path from n to a goal node, and is non-negative

#### Weighted A*:
Weighted A* is completely similar to A* but calculating f(n) is different. This difference will result in even faster search algorithm but the disadvantage can be missing the optimal path.

The idea of Weighted A* is speed up search at the expense of optimality Take an admissible heuristic, “inflate” it by a multiple α > 1, and then perform A* search as usual. Fewer nodes tend to get expanded, but the resulting solution may be suboptimal (its cost will be at most α times the cost of the optimal solution)

###  BFS vs IDS vs A*
We know that the complexity of FIFO queue is better than
the min-heap queue and we know that uninformed search algorithms are slower than
informed search algorithms. Also the time complexity of A* is better
than BFS and it is faster when we have a higher branching
factor. So:

We choose BFS if:
* time and memory is very critical
* our heuristic is not good and optimality and memory is important for us

We choose IDS if:
* the solution is in higher depth
* our heuristic is not good and optimality and memory is important for us

We choose A* if:
* we have an admissible heuristic



So each algorithm has it's own advantages and disadvantages so we should use them based on our problem properties.

BFS sometimes can have fewer explored states than A* and that's because it doesn't insert a state which is already in the frontier to frontier again, but A* has to do it, but usually it won't have to explore it so it's always faster.
#### As result, we conclude that informed search algorithms are much faster than uninformed algorithms.